<h1>Module 16 Project Template</h1>

Use the code below to help you analyze the data returned by the MBTA API:

In [1]:
mbtaURL = "https://api-v3.mbta.com/vehicles?filter[route]=1&include=trip"

import urllib.request, json
with urllib.request.urlopen(mbtaURL) as url:
    data = json.loads(url.read().decode())
   
    with open('data.json', 'w') as outfile:
        json.dump(data, outfile)
   
    with open('data.txt', 'w') as outfile:
        json.dump(json.dumps(data, indent=4, sort_keys=True), outfile)
       
    print(json.dumps(data, indent=4, sort_keys=True))


{
    "data": [
        {
            "attributes": {
                "bearing": 145,
                "carriages": [],
                "current_status": "IN_TRANSIT_TO",
                "current_stop_sequence": 16,
                "direction_id": 1,
                "label": "1923",
                "latitude": 42.34077304,
                "longitude": -71.08183496,
                "occupancy_status": "MANY_SEATS_AVAILABLE",
                "revenue": "REVENUE",
                "speed": null,
                "updated_at": "2025-04-05T11:19:30-04:00"
            },
            "id": "y1923",
            "links": {
                "self": "/vehicles/y1923"
            },
            "relationships": {
                "route": {
                    "data": {
                        "id": "1",
                        "type": "route"
                    }
                },
                "stop": {
                    "data": {
                        "id": "84",
                        "typ

Use the code below to read the data from your MySQL database and store it in a pandas dataframe.  

You you need to install pymysql to run the code:

pip install pymysql

In [2]:
import os
import pymysql
import pandas as pd

host = '127.0.0.1'
port = '3306'
user = 'root'
password = 'MyNewPass'
database = 'MBTAdb'

conn = pymysql.connect(
    host=host,
    port=int(3306),
    user="root",
    passwd=password,
    db=database,
    charset='utf8mb4')

df = pd.read_sql_query("SELECT * FROM mbta_buses",
    conn)

df.tail(10)

C:\Users\lbj20\AppData\Local\Temp\ipykernel_4288\2324256652.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM mbta_buses",


,record_num,id,trip_id,direction_id,label,stop_id,latitude,longitude,bearing,current_status,current_stop_sequence,occupancy_status,updated_at
224,297,y1802,66623884,0,1802,95,42.350555,-71.089229,343,IN_TRANSIT_TO,14,FULL,2025-04-05 11:19:14
225,298,y1923,66623978,1,1923,84,42.340947,-71.082051,133,IN_TRANSIT_TO,16,MANY_SEATS_AVAILABLE,2025-04-05 11:19:25
226,299,y1908,66623979,1,1908,75,42.360190,-71.095308,146,IN_TRANSIT_TO,10,FULL,2025-04-05 11:19:11
227,300,y1894,66623977,1,1894,187,42.341259,-71.082760,130,STOPPED_AT,15,MANY_SEATS_AVAILABLE,2025-04-05 11:19:26
228,301,y1880,66623975,1,1880,110,42.373042,-71.117460,287,STOPPED_AT,1,MANY_SEATS_AVAILABLE,2025-04-05 11:19:21
229,302,y1877,66623882,0,1877,108,42.370030,-71.112880,315,STOPPED_AT,22,MANY_SEATS_AVAILABLE,2025-04-05 11:18:55
230,303,y1850,66623974,1,1850,62,42.332735,-71.080689,307,IN_TRANSIT_TO,22,FEW_SEATS_AVAILABLE,2025-04-05 11:19:28
231,304,y1849,66623887,0,1849,6,42.332104,-71.079619,123,STOPPED_AT,4,MANY_SEATS_AVAILABLE,2025-04-05 11:19:08
232,305,y1833,66623885,0,1833,64,42.329840,-71.083930,180,STOPPED_AT,1,MANY_SEATS_AVAILABLE,2025-04-05 11:18:41
233,306,y1802,66623884,0,1802,95,42.350632,-71.089213,328,STOPPED_AT,14,FULL,2025-04-05 11:19:20



The code below saves the data as a csv file to the local directory as a backup.

In [3]:

df.to_csv('mbta.csv')

<h1>What is the average time it takes for a bus to complete the route</h1>

In [4]:
import pandas as pd
from haversine import haversine
import matplotlib.pyplot as plt

# 確保 updated_at 是 datetime 格式
df['updated_at'] = pd.to_datetime(df['updated_at'])

# 1️⃣ 計算每個 trip_id 的行程時間（秒）與平均
trip_durations = (
    df.groupby('trip_id')['updated_at']
    .agg(['min', 'max'])
    .reset_index()
)
trip_durations['duration_sec'] = (trip_durations['max'] - trip_durations['min']).dt.total_seconds()
trip_durations['duration_min'] = trip_durations['duration_sec'] / 60

average_duration = trip_durations['duration_min'].mean()
print(f"✅ 平均每班車完成時間：{average_duration:.2f} 分鐘")

# 2️⃣ 繪製一個 trip 的 stop_sequence 隨時間變化圖
# 選擇一個 trip_id 最完整的樣本
sample_trip_id = df['trip_id'].value_counts().idxmax()
trip_df = df[df['trip_id'] == sample_trip_id].sort_values(by='updated_at')

plt.plot(trip_df['updated_at'], trip_df['current_stop_sequence'], marker='o')
plt.title(f"Bus Progression Over Time (trip_id = {sample_trip_id})")
plt.xlabel("Time")
plt.ylabel("Current Stop Sequence")
plt.xticks(rotation=45)
plt.grid()
plt.tight_layout()
plt.show()

# 3️⃣ 計算該 trip 的平均速度
total_distance_km = 0
total_time_hr = 0

trip_df = trip_df.sort_values(by='updated_at').reset_index(drop=True)

for i in range(1, len(trip_df)):
    point1 = (trip_df.loc[i-1, 'latitude'], trip_df.loc[i-1, 'longitude'])
    point2 = (trip_df.loc[i, 'latitude'], trip_df.loc[i, 'longitude'])
    dist = haversine(point1, point2)
    time_diff = (trip_df.loc[i, 'updated_at'] - trip_df.loc[i-1, 'updated_at']).total_seconds() / 3600
    total_distance_km += dist
    total_time_hr += time_diff

avg_speed = total_distance_km / total_time_hr if total_time_hr > 0 else 0
print(f"✅ 平均車速（trip_id = {sample_trip_id}）：{avg_speed:.2f} km/h")


ModuleNotFoundError: No module named 'haversine'

<h1>Provide a Visualization based on the data</h1>

<h1>Give an estimate of the speed of the bus from current_stop_sequence = 1 to the last  current_stop_sequence. </h1> 

Note: You can use the haversine Python library to calculate the distance between two points, given two longitude and latitude coordinates.